# 提取交通地点的照片经纬度


In [26]:
import pandas as pd

import os

import exifread
from fractions import Fraction

## 从图像的EXIF信息中解析经纬度

In [27]:
def get_coordinates(file_path):
    '''
    输入图片文件路径，输出经纬度
    '''
    try:
        with open(file_path, 'rb') as f:
            tags = exifread.process_file(f)
            
            # 打印所有 EXIF 标签以进行调试
            print("EXIF 标签:", tags)
            
            # 检查是否存在 GPS 标签
            if 'GPS GPSLatitude' not in tags or 'GPS GPSLongitude' not in tags:
                print("缺少 GPS 信息")
                return None, None, None
            
            Latitude_list = tags['GPS GPSLatitude'].values
            Longitude_list = tags['GPS GPSLongitude'].values
            LatitudeRef = tags['GPS GPSLatitudeRef'].values
            LongitudeRef = tags['GPS GPSLongitudeRef'].values
            Time = tags.get('EXIF DateTimeOriginal', '未知时间').values
            
            # 检查纬度和经度列表的长度
            print("纬度列表:", Latitude_list)
            print("经度列表:", Longitude_list)
            
            if len(Latitude_list) < 3 or len(Longitude_list) < 3:
                print("纬度或经度数据不完整")
                return None, None, None
            
            # 处理纬度
            Latitude = convert_to_degrees(Latitude_list)
            # 根据纬度参考值确定纬度符号
            if LatitudeRef != 'N':
                Latitude = -Latitude
            
            # 处理经度
            Longitude = convert_to_degrees(Longitude_list)
            # 根据经度参考值确定经度符号
            if LongitudeRef != 'E':
                Longitude = -Longitude
            
            return Latitude, Longitude, Time
    except FileNotFoundError:
        print(f"文件未找到: {file_path}")
        return None, None, None
    except KeyError as e:
        print(f"缺少 EXIF 标签: {e}")
        return None, None, None
    except Exception as e:
        print(f"发生错误: {e}")
        return None, None, None

def convert_to_degrees(value):
    """将分数形式的 GPS 数据转换为度数"""
    # 处理度
    d = float(Fraction(value[0]))

    # 处理分
    if len(value) > 1:
        m = float(Fraction(value[1]))
    else:
        m = 0  # 如果没有分，设为 0

    # 处理秒
    if len(value) > 2:
        s = float(Fraction(value[2]))
    else:
        s = 0  # 如果没有秒，设为 0

    return d + (m / 60.0) + (s / 3600.0)

# # 调用函数
# result = get_coordinates('2024-11-02_154929.jpg')
# if result:
#     Latitude, Longitude, Time = result
#     print('纬度:', Latitude, '经度:', Longitude, '时间:', Time)
# else:
#     print("未能获取坐标信息")

## 遍历每张照片，获取所有照片经纬度信息

In [28]:
print(os.listdir('.'))

['.ipynb_checkpoints', '1.getLatitudeLongtitudeFromPhotos-Copy1.ipynb', '2.DrawMap-Copy1.ipynb', '2.DrawMap.ipynb', 'FileName&Infomation.csv', 'images', 'Step1getTransportaionLatitudeLongtitudeFromPhotos.ipynb', 'Transportation.csv', '拍照地点分布地图.html']


In [29]:
# 进入images目录
os.chdir('images/')
df_list = []  # 创建一个列表来存储每个文件的信息
print(os.listdir('.'))

['.ipynb_checkpoints', 'Transportation']


In [30]:


# for each in os.listdir():  # 遍历每个文件
#     if each == '.ipynb_checkpoints':  # 忽略'.ipynb_checkpoints'文件夹
#         continue
#     try:
#         coord = get_coordinates(each)
#         # 将每个文件的信息作为字典添加到列表中
#         df_list.append({'FileName': each, 'Latitude': coord[0], 'Longitude': coord[1], 'Time': coord[2]})
#     except Exception as e:
#         print('文件{}无法解析: {}'.format(each, e))

for folder_path, subfolders, filenames in os.walk('Transportation/'):
    for each in filenames:
        if each.endswith('.ipynb_checkpoints'):  # 忽略 '.ipynb_checkpoints' 文件
            continue
        file_path = os.path.join(folder_path, each)  # 获取文件的完整路径
        try:
            coord = get_coordinates(file_path)
            if coord is not None:
                # 将每个文件的信息作为字典添加到列表中
                df_list.append({
                    'FolderName': os.path.basename(folder_path),  # 添加文件夹名
                    'FileName': each,
                    'Latitude': coord[0],
                    'Longitude': coord[1],
                    'Time': coord[2]
                })
            else:
                print(f"文件 {each} 缺少 GPS 信息")
        except Exception as e:
            print(f'文件{each}无法解析: {e}')

# 最后将列表转换为 DataFrame
df = pd.DataFrame(df_list)

EXIF 标签: {'Image Make': (0x010F) ASCII=Apple @ 146, 'Image Model': (0x0110) ASCII=iPhone 13 Pro @ 152, 'Image Orientation': (0x0112) Short=Horizontal (normal) @ 42, 'Image XResolution': (0x011A) Ratio=72 @ 166, 'Image YResolution': (0x011B) Ratio=72 @ 174, 'Image ResolutionUnit': (0x0128) Short=Pixels/Inch @ 78, 'Image Software': (0x0131) ASCII=18.0.1 @ 182, 'Image DateTime': (0x0132) ASCII=2024:11:05 13:35:56 @ 190, 'Image HostComputer': (0x013C) ASCII=iPhone 13 Pro @ 210, 'Image ExifOffset': (0x8769) Long=224 @ 126, 'GPS GPSLatitudeRef': (0x0001) ASCII=N @ 2644, 'GPS GPSLatitude': (0x0002) Ratio=[22, 18, 207/20] @ 2820, 'GPS GPSLongitudeRef': (0x0003) ASCII=E @ 2668, 'GPS GPSLongitude': (0x0004) Ratio=[114, 10, 4091/100] @ 2844, 'GPS GPSAltitudeRef': (0x0005) Byte=0 @ 2692, 'GPS GPSAltitude': (0x0006) Ratio=61040/11539 @ 2868, 'GPS GPSTimeStamp': (0x0007) Ratio=[5, 35, 5511/100] @ 2876, 'GPS GPSSpeedRef': (0x000C) ASCII=K @ 2728, 'GPS GPSSpeed': (0x000D) Ratio=280618/116439 @ 2900, '

In [31]:
print(os.listdir('.'))

['.ipynb_checkpoints', 'Transportation']


In [32]:
df

,FolderName,FileName,Latitude,Longitude,Time
0,BusStation,Bus_Station_1.jpg,22.302875,114.178031,2024:11:05 13:35:56
1,BusStation,Bus_Station_2.jpg,22.302806,114.177917,2024:11:05 13:36:20
2,CrossHarbourTunnelBBI,Cross_Harbour_Tunnel_BBI_1.jpg,22.303436,114.180306,2024:11:05 13:56:43
3,CrossHarbourTunnelBBI,Cross_Harbour_Tunnel_BBI_2.jpg,22.302947,114.180511,2024:11:05 13:57:48
4,CrossHarbourTunnelBBI,Cross_Harbour_Tunnel_BBI_3.jpg,22.303283,114.180350,2024:11:05 13:57:56
5,HungHomStation,Hung_Hom_Station_1.jpg,22.303608,114.180497,2024:11:05 13:59:34
6,HungHomStation,Hung_Hom_Station_2.jpg,22.303672,114.180594,2024:11:05 14:00:03


In [33]:
# 返回上一级目录
os.chdir('../')
print(os.listdir('.'))

['.ipynb_checkpoints', '1.getLatitudeLongtitudeFromPhotos-Copy1.ipynb', '2.DrawMap-Copy1.ipynb', '2.DrawMap.ipynb', 'FileName&Infomation.csv', 'images', 'Step1getTransportaionLatitudeLongtitudeFromPhotos.ipynb', 'Transportation.csv', '拍照地点分布地图.html']


## 保存照片文件名及经纬度表格

In [34]:
df.to_csv('Transportation.csv', index=False, encoding='utf-8')